# Re-create Rachel Error

In [1]:
import os
import sys
import yaml
from subprocess import Popen
import pickle

import pandas as pd
import geopandas as gpd
import numpy as np


from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import WranglerLogger
from network_wrangler import ProjectCard

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import metcouncil

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

### Remote I/O and Parameters

In [4]:
# root_dir = os.path.join('C:\\', 'Users', 'david.ory', 'Documents', 'GitHub', 'client_met_council_network')
root_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'client_met_council_network')
input_dir = os.path.join(root_dir, 'network_standard')
pickle_dir = input_dir
output_dir = input_dir
card_dir = os.path.join(root_dir, 'project_cards')
# lasso_dir = os.path.join('C:\\', 'Users', 'david.ory', 'Documents', 'GitHub', 'lasso')
lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'lasso')

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2020-10-20 15:22:17, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso
2020-10-20 15:22:17, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso


### Read Version 00 Scenario

In [6]:
version_00_pickle_file_name = os.path.join(pickle_dir, 'working_scenario_00.pickle')
v_01_scenario = pickle.load(open(version_00_pickle_file_name, 'rb'))

### Compute Number of Lanes 

In [7]:
v_01_scenario.road_net.links_df = v_01_scenario.road_net.links_df.drop('lanes', axis = 1)
v_01_scenario.road_net.links_df.columns

Index(['model_link_id', 'osm_link_id', 'shstReferenceId', 'shstGeometryId',
       'shape_id', 'u', 'v', 'A', 'B', 'locationReferences', 'distance',
       'roadway', 'name', 'ref', 'bridge', 'tunnel', 'width', 'max_speed',
       'bike_facility', 'drive_access', 'walk_access', 'bike_access',
       'truck_access', 'bus_only', 'rail_only', 'access', 'price',
       'trn_priority', 'ttime_assert', 'geometry'],
      dtype='object')

In [8]:
v_01_scenario.road_net = metcouncil.calculate_number_of_lanes(
        roadway_net=v_01_scenario.road_net,
        parameters=parameters,
)
v_01_scenario.road_net.links_df.lanes.value_counts()

2020-10-20 15:22:32, INFO: Calculating Number of Lanes as network variable: 'lanes'
2020-10-20 15:22:32, INFO: Calculating Number of Lanes as network variable: 'lanes'
2020-10-20 15:22:32, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso
2020-10-20 15:22:32, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso
2020-10-20 15:22:55, INFO: Finished calculating number of lanes to: lanes
2020-10-20 15:22:55, INFO: Finished calculating number of lanes to: lanes


1.0    301047
2.0     26560
3.0      1734
4.0       545
5.0        66
6.0         6
Name: lanes, dtype: int64

### Apply Project Cards

In [9]:
project_card_list = []
card_01_file_name = os.path.join(card_dir, 'test_35W_ML.yml')

card = ProjectCard.read(card_01_file_name, validate = False)
project_card_list.append(card)


In [10]:
for project_card in project_card_list:
    v_01_scenario.apply_project(project_card)

2020-10-20 15:22:55, INFO: Applying 35W south managed lane
2020-10-20 15:22:55, INFO: Applying 35W south managed lane
2020-10-20 15:22:55, INFO: Applying Project to Roadway Network: 35W south managed lane
2020-10-20 15:22:55, INFO: Applying Project to Roadway Network: 35W south managed lane


/Users/wsp/opt/anaconda3/envs/lasso_x/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [11]:
v_01_scenario.applied_projects

['35W south managed lane']

### Make Travel Model Networks (for manual fixes)

In [12]:
m_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = v_01_scenario.road_net, 
    parameters = parameters)

2020-10-20 15:23:55, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso
2020-10-20 15:23:55, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso


In [13]:
# need to recalculate centroid connectors from lanes calculation
m_net.calculate_centroidconnect(overwrite=True)

2020-10-20 15:23:55, INFO: Overwriting existing Centroid Connector Variable 'centroidconnect' already in network
2020-10-20 15:23:55, INFO: Overwriting existing Centroid Connector Variable 'centroidconnect' already in network
2020-10-20 15:23:55, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2020-10-20 15:23:55, INFO: Calculating Centroid Connector and adding as roadway network variable: centroidconnect
2020-10-20 15:23:55, INFO: Finished calculating centroid connector variable: centroidconnect
2020-10-20 15:23:55, INFO: Finished calculating centroid connector variable: centroidconnect


In [14]:
m_net.links_df.columns

Index(['model_link_id', 'osm_link_id', 'shstReferenceId', 'shstGeometryId',
       'shape_id', 'u', 'v', 'A', 'B', 'locationReferences', 'distance',
       'roadway', 'name', 'ref', 'bridge', 'tunnel', 'width', 'max_speed',
       'bike_facility', 'drive_access', 'walk_access', 'bike_access',
       'truck_access', 'bus_only', 'rail_only', 'access', 'price',
       'trn_priority', 'ttime_assert', 'geometry', 'centroidconnect', 'lanes',
       'managed', 'ML_lanes', 'ML_price', 'segment_id', 'ML_access',
       'ML_egress'],
      dtype='object')

In [15]:
m_net.roadway_standard_to_met_council_network()

2020-10-20 15:23:55, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2020-10-20 15:23:55, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2020-10-20 15:23:55, INFO: Creating managed lane network.
2020-10-20 15:23:55, INFO: Creating managed lane network.
2020-10-20 15:23:55, INFO: Creating network with duplicated managed lanes
2020-10-20 15:23:55, INFO: Creating network with duplicated managed lanes


/Users/wsp/opt/anaconda3/envs/lasso_x/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
/Users/wsp/opt/anaconda3/envs/lasso_x/lib/python3.7/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
/Users/wsp/opt/anaconda3/envs/lasso_x/lib/python3.7/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

2020-10-20 15:24:04, INFO: Creating calculated roadway variables.
2020-10-20 15:24:04, INFO: Creating calculated roadway variables.
2020-10-20 15:24:04, INFO: Calculating Area Type from Spatial Data and adding as roadway network variable: area_type
2020-10-20 15:24:04, INFO: Calculating Area Type from Spatial Data and adding as roadway network variable: area_type


/Users/wsp/Documents/GitHub/Lasso/lasso/roadway.py:427: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2020-10-20 15:24:53, INFO: Finished Calculating Area Type from Spatial Data into variable: area_type
2020-10-20 15:24:53, INFO: Finished Calculating Area Type from Spatial Data into variable: area_type
2020-10-20 15:24:53, INFO: Adding roadway network variable for county using a spatial join with: /Users/wsp/Documents/GitHub/lasso/metcouncil_data/county/cb_2017_us_county_5m.shp
2020-10-20 15:24:53, INFO: Adding roadway network variable for county using a spatial join with: /Users/wsp/Documents/GitHub/lasso/metcouncil_data/county/cb_2017_us_county_5m.shp


/Users/wsp/Documents/GitHub/Lasso/lasso/roadway.py:292: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2020-10-20 15:25:32, INFO: Finished Calculating county variable: county
2020-10-20 15:25:32, INFO: Finished Calculating county variable: county
2020-10-20 15:25:33, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2020-10-20 15:25:33, INFO: Centroid Connector Variable 'centroidconnect' already in network. Returning without overwriting.
2020-10-20 15:25:33, INFO: Calculating MPO as roadway network variable: mpo
2020-10-20 15:25:33, INFO: Calculating MPO as roadway network variable: mpo
2020-10-20 15:25:33, INFO: Finished calculating MPO variable: mpo
2020-10-20 15:25:33, INFO: Finished calculating MPO variable: mpo
2020-10-20 15:25:33, INFO: Calculating Assignment Group and Roadway Class as network variables: 'assign_group' and 'roadway_class'
2020-10-20 15:25:33, INFO: Calculating Assignment Group and Roadway Class as network variables: 'assign_group' and 'roadway_class'
2020-10-20 15:25:33, INFO: Centroid Connector Variable 'centro